In [45]:
#import necessary libraries


import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.feature import hog,local_binary_pattern
import glob
import numpy as np
from sklearn import svm
from skimage import feature
from sklearn.metrics import classification_report
from skimage.color import rgb2gray

In [46]:
#unzip Spoof_data.zip


In [47]:
# import training data

live_train = glob.glob('Spoof_data/Training Biometrika Live/live/*.png')
spoof_train = glob.glob(r'Spoof_data/Training Biometrika Spoof/Training Biometrika Spoof/spoof/*.png') # negative

# import testing data

live_test = glob.glob(r'Spoof_data/Testing Biometrika Live/live/*.png') # positive
spoof_test = glob.glob(r'Spoof_data/Testing Biometrika Spoof/Testing Biometrika Spoof/spoof/*.png') # negative

In [48]:
# initialize empty lists to append data
live_train_images = []
spoof_train_images = []
live_test_images = []
spoof_test_images = []

# read images from given path and append to the variavles
for path in live_train: live_train_images.append(imread(path))
for path in spoof_train: spoof_train_images.append(imread(path))
for path in live_test: live_test_images.append(imread(path))
for path in spoof_test: spoof_test_images.append(imread(path))

In [49]:
live_train_images

[array([[[ 64,  64,  64],
         [ 62,  62,  62],
         [ 61,  61,  61],
         ...,
         [ 53,  53,  53],
         [ 56,  56,  56],
         [ 74,  74,  74]],
 
        [[ 67,  67,  67],
         [ 63,  63,  63],
         [ 62,  62,  62],
         ...,
         [ 56,  56,  56],
         [ 61,  61,  61],
         [ 76,  76,  76]],
 
        [[ 67,  67,  67],
         [ 66,  66,  66],
         [ 65,  65,  65],
         ...,
         [ 57,  57,  57],
         [ 58,  58,  58],
         [ 76,  76,  76]],
 
        ...,
 
        [[140, 140, 140],
         [139, 139, 139],
         [139, 139, 139],
         ...,
         [105, 105, 105],
         [113, 113, 113],
         [140, 140, 140]],
 
        [[139, 139, 139],
         [139, 139, 139],
         [135, 135, 135],
         ...,
         [108, 108, 108],
         [113, 113, 113],
         [142, 142, 142]],
 
        [[140, 140, 140],
         [138, 138, 138],
         [139, 139, 139],
         ...,
         [108, 108, 108],
  

In [50]:
# convert images to np arrays
live_train_images, spoof_train_images = np.asarray(live_train_images), np.asarray(spoof_train_images)
live_test_images, spoof_test_images = np.asarray(live_test_images), np.asarray(spoof_test_images)

In [44]:
# change the type of feature you want to use here
feature_type = "lbp" #hog or lbp

# initialize empty lists to append features
live_train_features, spoof_train_features = [], []
live_test_features, spoof_test_features = [], []

if(feature_type == "hog"):

    # calculate and store hog features of training data
    for live_img, spoof_img in zip(live_train_images, spoof_train_images):
        live_train_features.append(hog(live_img, feature_vector = True))
        spoof_train_features.append(hog(spoof_img, feature_vector = True))

    # calculate and store hog features of testing data
    for live_img, spoof_img in zip(live_test_images, spoof_test_images):
        live_test_features.append(hog(live_img, feature_vector = True))
        spoof_test_features.append(hog(spoof_img, feature_vector = True))

if(feature_type == "lbp"):

# parameters copied from scikit docs of lbp
    METHOD = 'uniform'
    R = 3
    P = 8 * R

    # calculate and store lbp features of training data
    for live_img, spoof_img in zip(live_train_images, spoof_train_images):
        live_img = rgb2gray(live_img)
        spoof_img = rgb2gray(spoof_img)
        live_train_features.append(local_binary_pattern(live_img, P, R, METHOD).flatten())
        spoof_train_features.append(local_binary_pattern(spoof_img, P, R, METHOD).flatten())

    # calculate and store lbp features of training data
    for live_img, spoof_img in zip(live_test_images, spoof_test_images):
        # convert rgb to gray
        live_img = rgb2gray(live_img)
        # convert rgb to gray
        spoof_img = rgb2gray(spoof_img)
        live_test_features.append(local_binary_pattern(live_img, P, R, METHOD).flatten())
        spoof_test_features.append(local_binary_pattern(spoof_img, P, R, METHOD).flatten())

In [51]:
# define labels 1 for live and 0 for spoof
# i.e. create a list of 200 1's and 200 0's
labels_train = [[1]]*len(live_train_features) + [[0]]*len(spoof_train_features)

# merge both positive and negative training data
full_train = live_train_features + spoof_train_features

# assign labels for all test data, 1 for positive, 0 for negative
labels_test = [[1]]*len(live_test_features) + [[0]]*len(spoof_test_features)

# merge both positive and negative training data
full_test = live_test_features + spoof_test_features

In [53]:
clf = svm.SVC()
features_train = np.array(full_train)
features_test = np.array(full_test)

# stack features and labels together
data_frame_train = np.hstack((features_train,labels_train))

# shuffle train data
np.random.shuffle(data_frame_train)

In [55]:
x_train = data_frame_train[:len(data_frame_train),:-1]
y_train = data_frame_train[:len(data_frame_train),-1]

# ready test data and labels
x_test = features_test
y_test = labels_test

In [56]:
clf.fit(x_train,y_train)

SVC()

In [57]:
y_pred = clf.predict(x_test)

In [61]:
# calculate other metrics
print(classification_report(y_test, y_pred))

# hog: (0.859375, 0.845, 0.8434343434343435, None)
# lbp: (0.9303430011968119, 0.9225, 0.9221452744065148, None)

              precision    recall  f1-score   support

           0       0.89      0.98      0.94       200
           1       0.98      0.88      0.93       200

    accuracy                           0.93       400
   macro avg       0.94      0.93      0.93       400
weighted avg       0.94      0.93      0.93       400

